In [ ]:
import scipy as sp
import scipy.io
import os
import numpy as np
import pandas as pd
import glob
import csv
import random as rand
from tqdm import tnrange, tqdm_notebook
from collections import Iterable
import matplotlib.pylab as plt
import matplotlib as mpl
import random as rand
from ipywidgets import *
import colorlover as cl
from scipy import stats
import importlib
import sys
sys.path.append(os.getcwd()+'\\..')
from utils import utils, zscores
from utils import plotting_utils as pu
from utils import auc_methods as am
import matplotlib.patches as patches
from matplotlib import gridspec
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize

%load_ext autoreload
%autoreload 2
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

In [ ]:
size_mult = 1
plt.close('all')

font = {'family' : 'Arial',
        'weight' : 'normal',
        'size'   : 12}

mpl.rc('font', **font)
mpl.rc('xtick', labelsize=12) 
mpl.rc('ytick', labelsize=12)
mpl.rc('axes', labelsize=12)

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42


In [ ]:
size_mult = 0.1
plt.close('all')

font = {'family' : 'Arial',
        'weight' : 'normal',
        'size'   : 6}

mpl.rc('font', **font)
mpl.rc('xtick', labelsize=6) 
mpl.rc('ytick', labelsize=6)
mpl.rc('axes', labelsize=6)

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42


In [ ]:
bin_size = 0.025
window = [-1, 3]
edges = np.arange(window[0], window[1], bin_size)

# Load data and prepare data
## Load behavior and spike data

In [ ]:
DATA_DIR = os.path.abspath('../data')
log_df = pd.read_hdf(os.path.join(DATA_DIR, 'log_df_processed_02-28-2019.h5'), 'fixed')
unit_key_df = pd.read_hdf(os.path.join(DATA_DIR, 'unit_key_df_processed_02-28-2019.h5'), 'fixed')

### Remove any units that were recorded from tactile lick-left/visual lick-right mice

In [ ]:
unit_key_df = unit_key_df[~(unit_key_df['mouse_name'].isin(['EF0083','EF0085', 'EF0112', 'EF0111']))] # these are mice with tetrodes not implanted in pricipal whisker
unit_key_df['mouse_name'].unique()

In [ ]:
def plot_raster(ax,ras,trial_total, window, bin_size = 0.025):
    """
    plot raster for a specific trial_type/outcome stacked on previous plotted raster
    """  
    spike_counts = []
    for trial, spike in enumerate(ras['spike_times(stim_aligned)']):
        spike = spike[(spike>window[0]) & (spike<=window[1])]
        ax.vlines(spike, trial + trial_total - .5, trial + trial_total +.5)
        ax.vlines(ras.iloc[trial]['first_lick'], trial + trial_total - .5, trial +
                   trial_total + .5, color = 'r', linewidth = 5)

        spike = spike[(spike>window[0]) & (spike<=window[1])]
        edges = np.arange(window[0], window[1]+bin_size*2, bin_size)
        count, _ = np.histogram(spike,edges)
        spike_counts.append(count)
    return spike_counts

In [ ]:
def plot_psth(spike_counts, ax, window, color, bin_size = 0.025):
    """
    plot psth corresponding to a raster
    """
    average_hist = np.mean(spike_counts, axis=0)/bin_size
    SE_hist = stats.sem(spike_counts)/bin_size
    edges = np.arange(window[0], window[1]+bin_size*2, bin_size)
    ax.plot(edges[0:-1], average_hist, color = color)
    ax.fill_between(edges[0:-1], average_hist-SE_hist, average_hist+SE_hist, alpha = 0.3, color = color)

In [ ]:
def plot_rasters_psths(rasters, window, ax, ax1, patch_ax, colors, bin_size = 0.025):
    """
    plot rasters and psths for each trial_type/outcome in "rasters"
    """
    
    trial_total = 1
    patch_ax.plot([0,2],[trial_total-.5, trial_total-.5], '--k')

    for i, ras in enumerate(rasters):
        spike_counts = plot_raster(ax, ras, trial_total, window)
        num_trials_in_raster = len(ras)
        patch_ax.add_patch(patches.Rectangle((0,trial_total -.5), 1,
                                    num_trials_in_raster+1, facecolor = colors[i], alpha = 0.5))

        patch_ax.plot([0,2],[trial_total-.5, trial_total-.5], '--k')
        plot_psth(spike_counts, ax1, window, color = colors[i], bin_size = bin_size)
        trial_total = trial_total + num_trials_in_raster +1

    return spike_counts, trial_total

In [ ]:
def plot_raster_fig(rasters, trial_type, window, colors, bin_size = 0.025, ylim_p = None):
    """
    setup figure and plot rasters and PSTHs of indicated unit
    """

    fig = plt.figure(figsize=(4, 3.5))
    gs = [gridspec.GridSpec(1,1) for i in range(4)]
    gs[0].update(bottom = 0.88, top=0.95, left = 0.2, right = 0.83)
    gs[2].update(bottom=0.15, top=0.41, left = 0.2, right = 0.83)
    gs[1].update(bottom=0.45, top=0.88, left = 0.2, right = 0.83)
    gs[3].update(bottom=0.45, top=0.88, left = 0.83, right = 0.9)

    (ax1, ax2, ax3) = [plt.subplot(gs[i][0, 0]) for i in range(3)]
    patch_ax = plt.subplot(gs[3][0, 0], sharey = ax2)

    ax1.add_patch(patches.Rectangle((0,0), 0.15, 0.75, facecolor = 'k', alpha = 0.75))
    spike_counts, trial_total = plot_rasters_psths(rasters, window, ax2,ax3, patch_ax, colors)

    for ax in [ax1,ax2,ax3, patch_ax]:
        ax.set_xlim(window[0],window[1]-bin_size)
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)

    ax1.axis('off')
    ax1.set_ylim(0,2)

    ax2.spines['bottom'].set_visible(False)
    ax2.set_ylabel('Trials')    
    ax2.axes.get_xaxis().set_ticks([])
    ax2.set_ylim(-1, trial_total+.5)

    og_ylim = ax2.get_ylim()
    ax2.spines['left'].set_bounds(0, trial_total)
    ax2.set_yticks(np.arange(0, og_ylim[1], 20))
        
    ax3.set_xlabel('Time(s)')
    ax3.set_ylabel('Firing\nrate (Hz)') 
    if ylim_p != None:
        ax3.set_ylim(ylim_p)

    patch_ax.axis('off')
    patch_ax.set_xlim(0,2)
    return fig

In [ ]:
def plot_unit(unit_name, log_df, x_min, x_max, trial_type = 'Stim_Som_NoCue', colors = ['k', 'C0'], ylim=None, bin_size = 0.025):
    """
    indexes trials by trial type and passes them to
    plot_rasters for plotting
    """
    if 'Som' in trial_type:
        resp = 1
    else:
        resp = 2
    
    current_cell = log_df[log_df['uni_id'] == unit_name]
    lick = current_cell[(current_cell['trial_type'] == trial_type) & current_cell['response'].isin([resp])]
    no_lick = current_cell[(current_cell['trial_type'] == trial_type) & current_cell['response'].isin([0])]
    rasters = [no_lick, lick]

    fig = plot_raster_fig(rasters, trial_type, [x_min, x_max], colors, ylim_p = ylim, bin_size = bin_size)

    return fig

In [ ]:
%pdb

In [ ]:
fig = plot_unit('07903-22-164t2', log_df, -0.25, 0.75, trial_type = 'Stim_Som_NoCue', bin_size = bin_size)

In [ ]:
fig = plot_unit('07903-22-164t2', log_df, -0.25, 0.75, trial_type = 'Stim_Vis_NoCue', bin_size = bin_size)

In [ ]:
fig = plot_unit('07903-22-164t2', log_df, -0.25, 0.75, trial_type = '1CycStim_Som_NoCue', bin_size = bin_size)

In [ ]:
fig = plot_unit('07903-22-164t2', log_df, -0.25, 0.75, trial_type = '1CycStim_Vis_NoCue', bin_size = bin_size)

In [ ]:
import cmocean

In [ ]:
sub_map = unit_key_df
colors = cmocean.cm.thermal
columns = ['Touch Stim Lick(z_score)', 'Short Touch Stim Lick(z_score)',
           'Visual Stim Lick(z_score)', 'Short Visual Stim Lick(z_score)']
data = {}
for z_map in columns:
    data[z_map] = pd.DataFrame(np.stack(sub_map[z_map].dropna().as_matrix()))

# py.iplot(fig, filename= 'Hit_CR_Miss')


fig5c_1 = mpl.figure(figsize = (6,7))

gs1 = gridspec.GridSpec(1,1)
gs2 = gridspec.GridSpec(1,1)

gs1.update(bottom=0.1, top=0.68, left = 0.15, right = 0.9)
gs2.update(bottom = 0.69, top=0.87, left = 0.15, right = 0.9)


ax1 = mpl.subplot(gs1[0, 0])
ax2 = mpl.subplot(gs2[0, 0], sharex =ax1)
ax2.axis('off')
ax2.add_patch(patches.Rectangle((0,3.6), 0.15, 1, facecolor = 'C0', alpha = .5))
ax2.add_patch(patches.Rectangle((0,1.2), 0.15, 1, facecolor = 'C1', alpha = .5))

ax2.add_patch(patches.Rectangle((0,2.4), 0.05, 1, facecolor = 'c', alpha = .5))
ax2.add_patch(patches.Rectangle((0,0), 0.05, 1, facecolor = '#feb24c', alpha = .5))


ax2.set_ylim(0,4.7)

Col1 = data[columns[0]]
Col2 = data[columns[1]]
Col3 = data[columns[2]]
Col4 = data[columns[3]]
# Col3 = data[columns[2]]

cols = [Col1, Col2, Col3, Col4]

m_psth = []
sem_psth = []

for z_map in cols:
    m_psth.append(z_map.mean(axis = 0))
    sem_psth.append(scipy.stats.sem(z_map, axis = 0))

colors = ['C0','c', 'C1','#feb24c']

xvals = np.arange(-1,3,bin_size)
for col in range(4):
    ax1.plot(xvals[:-1],m_psth[col], colors[col])
    ax1.fill_between(xvals[0:-1], m_psth[col]-sem_psth[col], m_psth[col]+sem_psth[col],
                     alpha=0.5,color = colors[col])
  
  
# ax1.text(.8, .8, "Hit" , size=18, transform=ax1.transAxes, color = colors[0])
ax1.text(.6, .96, "Touch lick" , size=18, transform=ax1.transAxes, color = colors[0])
ax1.text(.6, .9, "Short touch lick" , size=18, transform=ax1.transAxes, color = colors[1])
ax1.text(.6, .84, "Visual lick" , size=18, transform=ax1.transAxes, color = colors[2])
ax1.text(.6, .78, "Short visual lick" , size=18, transform=ax1.transAxes, color = colors[3])
ax1.set_xlabel('Time from stim onset (s)')
ax1.set_ylabel('Mean Z-score')
ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax1.set_xlim(-.5,.75)

fig5c_1

### load auc data

In [ ]:
tactile_lick_noLick_aucs = pd.read_hdf(r'C:\Users\efink\Documents\DATA\Crossmodal_only\Touch_lick_no_lick_auc_2019-02-27.h5', 'table')
visual_lick_noLick_aucs = pd.read_hdf(r'C:\Users\efink\Documents\DATA\Crossmodal_only\Visual_lick_no_lick_auc_2019-02-27.h5', 'table')

In [ ]:
visual_lick_noLick_aucs['mouse_name'].drop_duplicates().values

### filter auc data for tactile lick-right/ visual lick-left  data 

In [ ]:
tactile_lick_noLick_aucs = tactile_lick_noLick_aucs.merge(unit_key_df['uni_id'].to_frame(), on = 'uni_id', how = 'right')
visual_lick_noLick_aucs = visual_lick_noLick_aucs.merge(unit_key_df['uni_id'].to_frame(), on = 'uni_id', how = 'right')
visual_lick_noLick_aucs.shape

# Identify units with a significant detect probability (DP) and define onset times

In [ ]:
auc_col_names = tactile_lick_noLick_aucs.columns[tactile_lick_noLick_aucs.columns.str.contains('auc')]
low_conf_col_names = tactile_lick_noLick_aucs.columns[tactile_lick_noLick_aucs.columns.str.contains('low')]
up_conf_col_names =tactile_lick_noLick_aucs.columns[tactile_lick_noLick_aucs.columns.str.contains('up')]

In [ ]:
# find bins where auc is significant 
raw_tac_sig_AUC = pd.DataFrame(~((tactile_lick_noLick_aucs.loc[:,low_conf_col_names].fillna(0.5) >= 0.5).values & 
                    (tactile_lick_noLick_aucs.loc[:,up_conf_col_names].fillna(0.5) <= 0.5).values)*1, index = tactile_lick_noLick_aucs['uni_id'])

raw_vis_sig_AUC = pd.DataFrame(~((visual_lick_noLick_aucs.loc[:,low_conf_col_names].fillna(0.5) >= 0.5).values & 
                    (visual_lick_noLick_aucs.loc[:,up_conf_col_names].fillna(0.5) <= 0.5).values)*1, index = visual_lick_noLick_aucs['uni_id'])

# identify direction of significant bins
auc_dir_t = (tactile_lick_noLick_aucs.loc[:,auc_col_names] >= 0.5)*1 + (tactile_lick_noLick_aucs.loc[:,auc_col_names] < 0.5)*-1
auc_dir_v = (visual_lick_noLick_aucs.loc[:,auc_col_names] >= 0.5)*1 + (visual_lick_noLick_aucs.loc[:,auc_col_names] < 0.5)*-1
auc_dir_v = (visual_lick_noLick_aucs.loc[:,auc_col_names] >= 0.5)*1 + (visual_lick_noLick_aucs.loc[:,auc_col_names] < 0.5)*-1
auc_dir_t.index = tactile_lick_noLick_aucs['uni_id']; auc_dir_t.columns = raw_tac_sig_AUC.columns
auc_dir_v.index = visual_lick_noLick_aucs['uni_id']; auc_dir_v.columns = raw_vis_sig_AUC.columns

raw_tac_sig_AUC = raw_tac_sig_AUC*auc_dir_t
raw_vis_sig_AUC = raw_vis_sig_AUC*auc_dir_v

In [ ]:
# take running sum of bins to identify bins with consecutive significant bins
smooth_tac_sig_AUC = raw_tac_sig_AUC.apply(lambda y: np.convolve(y, [1,1], 'same'), axis = 1)
smooth_vis_sig_AUC = raw_vis_sig_AUC.apply(lambda y: np.convolve(y, [1,1], 'same'), axis = 1)
smooth_tac_sig_AUC = pd.DataFrame(smooth_tac_sig_AUC.to_dict()).T
smooth_vis_sig_AUC= pd.DataFrame(smooth_vis_sig_AUC.to_dict()).T

### to get the first bin of 2 that are significantly different from 0 need to subtract 1 from all_sig_tac/vis since convolve 
### was used with 'same' setting (otherwise would need to subtract 2 if setting was 'full')
all_sig_tac = (smooth_tac_sig_AUC.isin([2,-2])*1).apply(lambda y: np.where(y), axis = 1)
all_sig_tac = all_sig_tac.apply(lambda y: (y[0]-1)*bin_size-1 if len(y[0])>0 else [])

all_sig_vis = (smooth_vis_sig_AUC.isin([2,-2])*1).apply(lambda y: np.where(y), axis = 1)
all_sig_vis = all_sig_vis.apply(lambda y: (y[0]-1)*bin_size-1 if len(y[0])>0 else [])

### find first significant bin for each unit for each modality
all_first_sig_tac = all_sig_tac.apply(lambda y:  y[(y>=0) & (y<1)] if len(y)!=0 else []) 
all_first_sig_tac = all_first_sig_tac.apply(lambda y:  np.nan if len(y)==0 else np.min(y))

all_first_sig_vis = all_sig_vis.apply(lambda y:  y[(y>=0) & (y<1)] if len(y)!=0 else [])
all_first_sig_vis = all_first_sig_vis.apply(lambda y:  np.nan if len(y)==0 else np.min(y))
dp_onsets = pd.merge(all_first_sig_vis.to_frame(name = 'vis_dp_onset').reset_index(), all_first_sig_tac.to_frame(name = 'tac_dp_onset').reset_index(), how = 'outer')
dp_onsets = dp_onsets.rename(columns = {'index': 'uni_id'})

# Find good example units that show range of responses

## sort units either by tac/vis auc ratio or mean auc

In [ ]:
visual_lick_noLick_aucs.shape

In [ ]:
m_tac_cp = tactile_lick_noLick_aucs.loc[:, auc_col_names[39:50]].mean(axis = 1)
m_vis_cp = visual_lick_noLick_aucs.loc[:, auc_col_names[39:50]].mean(axis = 1)

ratio = m_vis_cp.values/m_tac_cp.values
ratio_ind = np.argsort(ratio)

high_mean = m_vis_cp.values*m_tac_cp.values
high_mean_ind = np.argsort(high_mean)[::-1]
                     
sorted_ratio_uni_ids = tactile_lick_noLick_aucs.iloc[ratio_ind]['uni_id']
sorted_mean_uni_ids = tactile_lick_noLick_aucs.iloc[high_mean_ind]['uni_id']

### generators that make flipping through units easy
ratio_unit_gen = (unit for unit in sorted_ratio_uni_ids)
high_mean_unit_gen = (unit for unit in sorted_mean_uni_ids)

In [ ]:
### convert onsets to bin indeces
dp_onsets[['tac_dp_onset_inds', 'vis_dp_onset_inds']] = ((dp_onsets[['tac_dp_onset', 'vis_dp_onset']].fillna(-0)+1)/0.025).astype(int)

In [ ]:
am.plot_auc(next(high_mean_unit_gen), tactile_lick_noLick_aucs, visual_lick_noLick_aucs, window = [-.25,1])

In [ ]:
am.plot_auc('07603-04-162t1', tactile_lick_noLick_aucs, visual_lick_noLick_aucs, window = [-.25,1])

In [ ]:
am.plot_auc('07401-09-164t3', tactile_lick_noLick_aucs, visual_lick_noLick_aucs, window = [-0.25,1])

In [ ]:
am.plot_auc('07903-22-164t2', tactile_lick_noLick_aucs, visual_lick_noLick_aucs, window = [-0.25,1])

In [ ]:
dp_onsets

In [ ]:
dp_onsets

In [ ]:
## calculate mean length of dp
sig_tac_units = dp_onsets.dropna(subset = ['tac_dp_onset'])[['uni_id', 'tac_dp_onset_inds']].set_index('uni_id')
sig_tac_units['non_sig_bins'] = (raw_tac_sig_AUC.loc[sig_tac_units.index] == 0).apply(lambda x: np.where(x)[0], axis =1)
sig_tac_units['tac_dp_offset_inds'] = sig_tac_units.apply(lambda x:  x['non_sig_bins'][x['non_sig_bins']>x['tac_dp_onset_inds']][0], axis =1)

tac_mean = (sig_tac_units['tac_dp_offset_inds']-sig_tac_units['tac_dp_onset_inds']).mean()
tac_sem = (sig_tac_units['tac_dp_offset_inds']-sig_tac_units['tac_dp_onset_inds']).std()/np.sqrt(sig_tac_units.shape[0])

sig_vis_units = dp_onsets.dropna(subset = ['vis_dp_onset'])[['uni_id', 'vis_dp_onset_inds']].set_index('uni_id')
sig_vis_units['non_sig_bins'] = (raw_vis_sig_AUC.loc[sig_vis_units.index] == 0).apply(lambda x: np.where(x)[0], axis =1)
sig_vis_units['vis_dp_offset_inds'] = sig_vis_units.apply(lambda x:  x['non_sig_bins'][x['non_sig_bins']>x['vis_dp_onset_inds']][0], axis =1)

vis_mean = (sig_vis_units['vis_dp_offset_inds']-sig_vis_units['vis_dp_onset_inds']).mean()
vis_sem = (sig_vis_units['vis_dp_offset_inds']-sig_vis_units['vis_dp_onset_inds']).std()/np.sqrt(sig_vis_units.shape[0])

print(f"Vis dp duration: mean = {vis_mean}, sem = {vis_sem}")

In [ ]:
example_units = ['07603-04-162t1', '07401-09-164t3', '07903-22-164t2']

# Plot scatter plot of touch DP and vis DP and plot modality preference index
* use any unit that has at a significant DP in at least one modality trial.
* average over the three seconds after stimulus onset

In [ ]:
def plot_choice_prob_scatter(uni_ids, tac_auc_df, vis_auc_df, window, color, ax):
    tac = tac_auc_df.merge(uni_ids.to_frame(), on = 'uni_id')
    vis = vis_auc_df.merge(uni_ids.to_frame(), on = 'uni_id')
    tac['tac_cp'] = tac.iloc[:, window[0]:window[1]].mean(axis = 1)
    vis['vis_cp'] = vis.iloc[:, window[0]:window[1]].mean(axis = 1)
    ax.scatter(tac['tac_cp'], vis['vis_cp'], color = color, alpha = 0.3,clip_on=False, s = 10)
    ax.plot([0,1], [0,1], '--k')
    ax.plot([0,1], [0.5,0.5], 'k', alpha = 0.4)
    ax.plot([0.5,.5], [0,1],'k', alpha = 0.4)
    ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False) 
    ax.set_ylim(0.3,0.9), ax.set_xlim(0.3,0.9)
    ax.set_xlabel('Mean touch dp')
    ax.set_ylabel('Mean visual dp')
    return tac[['uni_id', 'tac_cp']], vis[['uni_id', 'vis_cp']]

In [ ]:
def calc_pref(mean_tac_dp, mean_vis_dp):
    pref = (abs(mean_tac_dp-0.5) - abs(mean_vis_dp-0.5))/(abs(mean_tac_dp-0.5) + abs(mean_vis_dp-0.5))
    return pref

In [ ]:
dp_onsets['tac_dp_onset'].dropna()

In [ ]:
dp_onsets[~dp_onsets[['vis_dp_onset', 'tac_dp_onset']].isnull().all(axis=1)]

In [ ]:
all_sig_units = dp_onsets.loc[~dp_onsets[['vis_dp_onset', 'tac_dp_onset']].isnull().all(axis = 1), 'uni_id'].to_frame()
all_sig_units.shape

In [ ]:
all_sig_units.shape

In [ ]:
fig, axes = plt.subplots(1,2, figsize = (10,3.5), gridspec_kw = {'width_ratios':[1, 2]})

mean_tac_cp, mean_vis_cp = plot_choice_prob_scatter(all_sig_units['uni_id'], tactile_lick_noLick_aucs, visual_lick_noLick_aucs, [45,159], 'k', axes[0])
for unit in example_units:
    x = mean_tac_cp.loc[mean_tac_cp['uni_id'] == unit, 'tac_cp'].values[0]
    y = mean_vis_cp.loc[mean_tac_cp['uni_id'] == unit, 'vis_cp'].values[0]
#     axes[0].annotate('local max', xy=(x, y), xytext=(3, 1.5),arrowprops=dict(facecolor='black', shrink=0.05))
    axes[0].scatter(x, y)
    print(x, y)
prefs = calc_pref(mean_tac_cp['tac_cp'], mean_vis_cp['vis_cp']).T
all_sig_units['prefs'] = prefs
                 
import ptitprince as pt
ax1 = axes[-1]
plt.sca(ax1)
pt.RainCloud(x = prefs, ax = ax1, width_viol=0.5, width_box=0.075, move = 0.2, palette = ['C7'], orient = 'h')
ax1.spines['top'].set_visible(False); ax1.spines['right'].set_visible(False)
ax1.set_ylabel('modality preference \n(touch cp - visual cp) / \n(touch cp + visual cp)')
fig.tight_layout()
# fig.savefig('Lick-units_scatter.pdf', format='pdf', dpi=900, transparent = True)

### Visual DP more often than not begins later than touch DP. Taking average of all 3 seconds after stim onset tries to control for that when comparing between trial types. However, this dilutes the average DP for both trials significantly and takes into account time bins that are not relevant for DP (ie >1.5 seconds - this is well beyond most of the reaction times of the mice). One solution is to align trials to the detected onsets for each unit in each trial type and then average for the first 500ms

In [ ]:
def align_auc(auc_df, onsets):
    auc_cols = ['uni_id'] + list(auc_df.columns[auc_df.columns.str.contains('auc')])
    auc_df = auc_df.loc[:, auc_cols]
    auc_df = auc_df.merge(onsets, on = 'uni_id')
    onset_column = onsets.columns[-1]
    aligned_auc = np.vstack(auc_df.apply(lambda x: [x[['uni_id']+x[onset_column]].values], axis = 1).values)
    aligned_auc_df = pd.DataFrame(aligned_auc[:,1:], index= aligned_auc[:,0])
    aligned_auc_df = aligned_auc_df.reset_index().rename(columns = {'index': 'uni_id'})
    return aligned_auc_df

In [ ]:
dp_onsets['tac_dp_auc_window']= dp_onsets['tac_dp_onset_inds'].apply(lambda x: ['auc_score'+str(i) for i in range(x, x+40)])
dp_onsets['vis_dp_auc_window']= dp_onsets['vis_dp_onset_inds'].apply(lambda x: ['auc_score'+str(i) for i in range(x, x+40)])

In [ ]:
aligned_tac_dp = align_auc(tactile_lick_noLick_aucs, dp_onsets[['uni_id','tac_dp_auc_window']])
aligned_vis_dp = align_auc(visual_lick_noLick_aucs, dp_onsets[['uni_id','vis_dp_auc_window']])

aligned_tac_dp = aligned_tac_dp.merge(dp_onsets[['uni_id', 'tac_dp_onset']])
aligned_vis_dp = aligned_vis_dp.merge(dp_onsets[['uni_id', 'vis_dp_onset']])

aligned_tac_dp.head()

In [ ]:
prefs.shape

In [ ]:
mean_tac_cp.shape

In [ ]:
fig, axes = plt.subplots(1,2, figsize = (11,3.7), gridspec_kw = {'width_ratios':[1, 2.5]})

mean_tac_cp, mean_vis_cp = plot_choice_prob_scatter(all_sig_units['uni_id'], aligned_tac_dp, aligned_vis_dp, [1,20], 'C7', axes[0])
for unit in example_units:
    x = mean_tac_cp.loc[mean_tac_cp['uni_id'] == unit, 'tac_cp'].values[0]
    y = mean_vis_cp.loc[mean_tac_cp['uni_id'] == unit, 'vis_cp'].values[0]
#     axes[0].annotate('local max', xy=(x, y), xytext=(3, 1.5),arrowprops=dict(facecolor='black', shrink=0.05))
    axes[0].scatter(x, y)
    print(x, y)
prefs = calc_pref(mean_tac_cp['tac_cp'], mean_vis_cp['vis_cp']).T
all_sig_units['prefs'] = prefs.values
example_prefs = all_sig_units[all_sig_units['uni_id'].isin(example_units)]

import ptitprince as pt
ax1 = axes[-1]
plt.sca(ax1)
pt.RainCloud(x = prefs, ax = ax1, width_viol=0.5, width_box=0.05, move = 0.25, palette = ['C7'], orient = 'h', jitter = .15)
ax1.scatter(example_prefs['prefs'], [-.1]*3, c = ['C1', 'C0', 'C2'])
ax1.axvline(x = 0, linestyle = '--', color = 'k')
ax1.spines['top'].set_visible(False); ax1.spines['right'].set_visible(False)
ax1.set_ylabel('modality preference index')
fig.tight_layout()


print(f"t-test, p = {scipy.stats.ttest_1samp(prefs, 0)[1]:.4f}")
fig.savefig('Lick-units_scatter.pdf', format='pdf', dpi=900, transparent = True)

In [ ]:
def cum_dist(arr, bins):
    hist = np.histogram(arr, bins = bins)
    cumsum = np.cumsum(hist[0])
    return [cumsum, hist[1]]

In [ ]:
def prettify_ax(ax, x_label, y_label, y_lim = None, x_lim = None):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    
    if y_lim:
        ax.set_ylim(y_lim)
        
    if x_lim:
        ax.set_xlim(x_lim)
        
    return ax

In [ ]:
figure, ax = plt.subplots(1,1,figsize = (9.5, 9))

cumsum = cum_dist(dp_onsets['tac_dp_onset'].dropna(), bins = np.arange(0,1, 0.025))
ax.plot(cumsum[1][:-1], cumsum[0]/dp_onsets['tac_dp_onset'].dropna().shape, color = 'm')

cumsum = cum_dist(dp_onsets['vis_dp_onset'].dropna(), bins = np.arange(0,1, 0.025))
ax.plot(cumsum[1][:-1], cumsum[0]/dp_onsets['vis_dp_onset'].dropna().shape, color = 'C1')

prettify_ax(ax, 'DP onset (s)', 'Cummulative fraction')

ticks = np.arange(0,1.05,0.05)
ax.set_xticks(ticks)
ticks = np.round(ticks.copy(),2).astype(str)
ticks[1::2] = ''
ax.set_xticklabels(ticks)

ticks = np.arange(0,1.05,0.05)
ax.set_yticks(ticks)
ticks = np.round(ticks.copy(),2).astype(str)
ticks[1::2] = ''
ax.set_yticklabels(ticks)

# figure.savefig('DP_onset_cum_dist.pdf', format = 'pdf', dpi = 900)

## ks test testing whether DP onset distributions are significantly different

In [ ]:
dp_onsets['tac_dp_onset'].dropna().shape

In [ ]:
dp_onsets['vis_dp_onset'].dropna().shape

In [ ]:
scipy.stats.ks_2samp(dp_onsets['tac_dp_onset'].dropna(), dp_onsets['vis_dp_onset'].dropna())

## Compare DP onset of neural data with DP onset for whisker tracking data from Rajan and Yi-ting's revision experiment

In [ ]:
surrogate_whisker_touch_dp = pd.read_csv('Surrogate_whisker_Tactile_DP_onsets.csv')
surrogate_whisker_visual_dp = pd.read_csv('Surrogate_whisker_Visual_DP_onsets.csv')

target_whisker_touch_dp = pd.read_csv('Target_whisker_Tactile_DP_onsets.csv')
target_whisker_visual_dp = pd.read_csv('Target_whisker_Visual_DP_onsets.csv')

In [ ]:
for df in [surrogate_whisker_touch_dp,surrogate_whisker_visual_dp, target_whisker_touch_dp, target_whisker_visual_dp]:
    df.loc[0, 'X'] = 0.0
    df.iloc[-1].loc['X'] = 1.0

    df.loc[0, 'X'] = 0.0
    df.iloc[-1].loc['X'] = 1.0

In [ ]:
tactile_dp_units = dp_onsets[['uni_id', 'tac_dp_onset']].dropna()
visual_dp_units  = dp_onsets[['uni_id', 'vis_dp_onset']].dropna()

In [ ]:
tactile_dp_units = tactile_dp_units.merge(unit_key_df[['uni_id','mouse_name', 'date', 'RT_median_TLR']], how = 'left')
visual_dp_units  = visual_dp_units.merge(unit_key_df[['uni_id','mouse_name', 'date', 'RT_median_VLL']], how = 'left')

In [ ]:
figure, (ax1, ax2) = plt.subplots(1,2,figsize = (17.5, 6.5))


for n, alpha, text_offset  in zip(range(1,5), [1,.8,.6,.4, .2], [0,.04,.08,.12,.16]):
    tactile_top_n_dp_units = (tactile_dp_units.set_index('uni_id')
                                     .groupby(['mouse_name','date'])
                                     .apply(lambda x: x.sort_values('tac_dp_onset').iloc[:n]['tac_dp_onset'])
                                     .reset_index())
    visual_top_n_dp_units = (visual_dp_units.set_index('uni_id')
                                    .groupby(['mouse_name','date'])
                                    .apply(lambda x:  x.sort_values('vis_dp_onset').iloc[:n]['vis_dp_onset'])
                                    .reset_index())


    tactile_top_n_dp_units.columns = ['mouse_name', 'date', 'uni_id', 'tac_dp_onset']
    visual_top_n_dp_units.columns = ['mouse_name', 'date', 'uni_id', 'vis_dp_onset']

    cumsum = cum_dist(tactile_top_n_dp_units['tac_dp_onset'], bins = np.arange(0,1, 0.025))
    ax1.plot(cumsum[1][:-1], cumsum[0]/tactile_top_n_dp_units['tac_dp_onset'].shape[0], color = 'm', alpha = alpha)
    
    ax1.text(.4, .45 - text_offset, 
             f'shortest {n} dp onsets per session (total n = {tactile_top_n_dp_units.shape[0]})'
             , color = 'm', alpha = alpha)

    cumsum = cum_dist(visual_top_n_dp_units['vis_dp_onset'], bins = np.arange(0,1, 0.025))
    ax2.plot(cumsum[1][:-1], cumsum[0]/visual_top_n_dp_units['vis_dp_onset'].shape[0], color = 'C1', alpha = alpha)

    ax2.text(.4, .45 - text_offset, 
         f'shortest {n} dp onsets per session (total n = {visual_top_n_dp_units.shape[0]})'
         , color = 'C1', alpha = alpha)

ax1.plot(surrogate_whisker_touch_dp['X'], surrogate_whisker_touch_dp['Y'],'k')
ax1.plot(target_whisker_touch_dp['X'], target_whisker_touch_dp['Y'], color = [0.35,0.35,0.35])
ax2.plot(surrogate_whisker_visual_dp['X'], surrogate_whisker_visual_dp['Y'], 'k')
ax2.plot(target_whisker_visual_dp['X'], target_whisker_visual_dp['Y'], color = [0.35,0.35,0.35])

ax1.set_title('Touch lick trials')
ax2.set_title('Visual lick trials')

ax1.text(0.37, 0.5, 'Neural activity DP onset', color = 'm')
ax1.text(0.37, 0.25, 'Surrogate Whisker angle DP onset', color = 'k')
ax1.text(0.37, 0.2, 'Target Whisker angle DP onset', color = [0.35,0.35,0.35])

ax2.text(0.37, 0.5, 'Neural activity DP onset', color = 'C1')
ax2.text(0.37, 0.25, 'Surrogate Whisker angle DP onset', color = 'k')
ax2.text(0.37, 0.2, 'Target Whisker angle DP onset', color = [0.35,0.35,0.35])



for ax in [ax1,ax2]:
    prettify_ax(ax, 'DP onset (s)', 'Cummulative fraction')
    ticks = np.arange(0,1.05,0.05)
    ax.set_xticks(ticks)
    ax.set_xlim(0,1)
    ax.set_ylim(0,1)
    ticks = np.round(ticks.copy(),2).astype(str)
    ticks[1::2] = ''
    ax.set_xticklabels(ticks)

    ticks = np.arange(0,1.05,0.05)
    ax.set_yticks(ticks)
    ticks = np.round(ticks.copy(),2).astype(str)
    ticks[1::2] = ''
    ax.set_yticklabels(ticks)
    
figure.savefig('DP_neural_vs_whisker.pdf', format='pdf', dpi=900, transparent = True)

In [ ]:
tactile_top_1_dp_units = (tactile_dp_units.set_index('uni_id')
                                 .groupby(['mouse_name','date'])
                                 .apply(lambda x: x.sort_values('tac_dp_onset').iloc[:1]['tac_dp_onset'])
                                 .reset_index())
visual_top_1_dp_units = (visual_dp_units.set_index('uni_id')
                                .groupby(['mouse_name','date'])
                                .apply(lambda x:  x.sort_values('vis_dp_onset').iloc[:1]['vis_dp_onset'])
                                .reset_index())

q3_tac, q1_tac = np.percentile(tactile_top_1_dp_units['tac_dp_onset'], [75 ,25])
iqr_tac = q3_tac - q1_tac

q3_vis, q1_vis = np.percentile(visual_top_1_dp_units['vis_dp_onset'], [75 ,25])
iqr_vis = q3_vis - q1_vis

print(f"median for shortest 1 dp distribution (tactile): {tactile_top_1_dp_units['tac_dp_onset'].median():.3} sec")
print(f"25th percentile (tactile): {q1_tac:.5} sec")
print(f"75th percentile (tactile): {q3_tac:.5} sec")
print(f"IQR shortest 1 dp distribution (tactile): {iqr_tac:.4} sec\n\n")


print(f"median for shortest 1 dp distribution (visual): {visual_top_1_dp_units['vis_dp_onset'].median():.3} sec")
print(f"25th percentile (visual): {q1_vis:.5} sec")
print(f"75th percentile (visual): {q3_vis:.5} sec")
print(f"IQR shortest 1 dp distribution (visual): {iqr_vis:.4} sec")



In [ ]:
surrogate_whisker_dp_indv = pd.read_csv('Surrogate_whisker_DP_onsets.csv')
median_surr_whisker_dp = surrogate_whisker_dp_indv['Tactile'].dropna().median()
q3_whisker, q1_whisker = np.percentile(surrogate_whisker_dp_indv['Tactile'].dropna(), [75 ,25])
iqr_whisker = q3_whisker - q1_whisker

print(f"median for surrogate whisker dp distribution (tactile): {median_surr_whisker_dp:.3} sec")
print(f"25th percentile (whisker): {q1_whisker:.5} sec")
print(f"75th percentile (whisker): {q3_whisker:.5} sec")
print(f"IQR surrogate whisker dp distribution (tactile): {iqr_whisker:.3} sec")


In [ ]:
q3_whisker - q1_whisker

In [ ]:
scipy.stats.ks_2samp(tactile_top_1_dp_units['tac_dp_onset'].dropna(), surrogate_whisker_dp_indv['Tactile'].dropna(), 'greater')

In [ ]:
scipy.stats.ks_2samp(visual_top_1_dp_units['vis_dp_onset'].dropna(), surrogate_whisker_dp_indv['Visual'].dropna())

In [ ]:
tactile_preRT_DP = tactile_dp_units[tactile_dp_units['tac_dp_onset']<tactile_dp_units['RT_median_TLR']]
visual_preRT_DP = visual_dp_units[visual_dp_units['vis_dp_onset']<visual_dp_units['RT_median_VLL']]

tac_dp_stats = tactile_top_quantile_dp_units.groupby(['mouse_name','date'])['uni_id'].count().describe(percentiles = [0.1,0.2,0.5,0.8, 0.9])
vis_dp_stats = visual_top_quantile_dp_units.groupby(['mouse_name','date'])['uni_id'].count().describe(percentiles = [0.1,0.2,0.5,0.8, 0.9])

summary_stats = pd.concat([tac_dp_stats, vis_dp_stats], axis = 1)
summary_stats.columns = ['Touch', 'Visual']
summary_stats

## plot figures describing numbers and fraction of neurons showing DP and SP

In [ ]:
touch_lick_units = dp_onsets.loc[~dp_onsets['tac_dp_onset'].isnull() & dp_onsets['vis_dp_onset'].isnull(), 'uni_id'].to_frame()
vis_lick_units = dp_onsets.loc[~dp_onsets['vis_dp_onset'].isnull() & dp_onsets['tac_dp_onset'].isnull(), 'uni_id'].to_frame()
bimodal_lick_units = dp_onsets.loc[~dp_onsets[['vis_dp_onset', 'tac_dp_onset']].isnull().any(axis = 1), 'uni_id'].to_frame()

In [ ]:
touch_lick_units.shape[0] + vis_lick_units.shape[0] + bimodal_lick_units.shape[0]

In [ ]:
def get_sig_auc(df, num_bins_sig = 2, time_offset = -1, sig_window = [0,1]):
    # find bins where auc is significant 
    auc_col_names = df.columns[df.columns.str.contains('auc')]
    low_conf_col_names = df.columns[df.columns.str.contains('low')]
    up_conf_col_names =df.columns[df.columns.str.contains('up')]

    raw_sig_AUC = pd.DataFrame(~((df.loc[:,low_conf_col_names].fillna(0.5) >= 0.5).values & 
                        (df.loc[:,up_conf_col_names].fillna(0.5) <= 0.5).values)*1, index = df['uni_id'])

    # identify direction of significant bins
    auc_dir = (df.loc[:,auc_col_names] >= 0.5)*1 + (df.loc[:,auc_col_names] < 0.5)*-1
    auc_dir.index = df['uni_id']; auc_dir.columns = raw_sig_AUC.columns
    raw_sig_AUC = raw_sig_AUC*auc_dir

    # take running sum of bins to identify bins with consecutive significant bins
    smooth_sig_AUC = raw_sig_AUC.apply(lambda y: np.convolve(y, [1]*num_bins_sig, 'same'), axis = 1)
    smooth_sig_AUC = pd.DataFrame(smooth_sig_AUC.to_dict()).T

    ### to get the first bin of 2 that are significantly different from 0 need to subtract 1 from all_sig_tac/vis since convolve 
    ### was used with 'same' setting (otherwise would need to subtract 2 if setting was 'full')
    all_sig = (smooth_sig_AUC.isin([num_bins_sig,-num_bins_sig])*1).apply(lambda y: np.where(y), axis = 1)
    all_sig = all_sig.apply(lambda y: (y[0]-1)*bin_size+time_offset if len(y[0])>0 else [])

    ### find first significant bin for each unit
    all_first_sig = all_sig.apply(lambda y:  y[(y>=sig_window[0]) & (y<sig_window[1])] if len(y)!=0 else []) 
    all_first_sig = all_first_sig.apply(lambda y:  np.nan if len(y)==0 else np.min(y))

    return all_first_sig


In [ ]:
touch_stim_prob_auc = pd.read_hdf(r'C:\Users\efink\Documents\DATA\Crossmodal_only\Touch_stim_prob_auc_2019-02-24.h5', 'table')
visual_stim_prob_auc = pd.read_hdf(r'C:\Users\efink\Documents\DATA\Crossmodal_only\Visual_stim_prob_auc_2019-02-28.h5', 'table')
touch_detect_prob_auc = pd.read_hdf(r'C:\Users\efink\Documents\DATA\Crossmodal_only\Touch_lick_no_lick_auc_2019-02-27.h5', 'table')
visual_detect_prob_auc = pd.read_hdf(r'C:\Users\efink\Documents\DATA\Crossmodal_only\Visual_lick_no_lick_auc_2019-02-27.h5', 'table')

In [ ]:
all_first_sig_tac_dp = get_sig_auc(touch_detect_prob_auc.merge(unit_key_df))
all_first_sig_vis_dp = get_sig_auc(visual_detect_prob_auc.merge(unit_key_df))

all_first_sig_tac_sp = get_sig_auc(touch_stim_prob_auc.merge(unit_key_df),num_bins_sig =2, time_offset=-0.25, sig_window = [0,0.200])


In [ ]:
touch_dp_units = set(all_first_sig_tac_dp.dropna().index)
visual_dp_units = set(all_first_sig_vis_dp.dropna().index)
touch_sp_units = set(all_first_sig_tac_sp.dropna().index)


In [ ]:
size_mult = 0.1

In [ ]:
from matplotlib_venn import venn3, venn3_circles

fig = plt.figure(figsize = [20*size_mult,20*size_mult])
v = venn3((touch_dp_units,visual_dp_units,touch_sp_units), ('Touch-Lick neurons', 'Visual-Lick neurons', 'Touch-stim neurons'), set_colors = ['#7F3F97', 'C1', 'cyan'], alpha = 0.5 )
venn3_circles((touch_dp_units,visual_dp_units,touch_sp_units))
# v.get_patch_by_id('10').set_color('#7F3F97')
# v.get_patch_by_id('01').set_color('C1')
fig.savefig('modality_venn.pdf', format = 'pdf', dpi = 900)

In [ ]:
mean_sp = touch_stim_prob_auc.set_index('uni_id').loc[:, auc_col_names[9:17]].apply(np.mean, axis = 1)

In [ ]:
mean_cp = mean_tac_cp.merge(mean_vis_cp, how = 'outer').set_index('uni_id')

In [ ]:
pos_tac_dp = mean_cp.loc[touch_dp_units, 'tac_cp']>=0.5
pos_vis_dp = mean_cp.loc[visual_dp_units, 'vis_cp']>=0.5
pos_tac_sp = mean_sp.loc[touch_sp_units]>=0.5


In [ ]:
size_mult = .7

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (14*size_mult,12*size_mult))

for i, group in enumerate([pos_tac_dp, pos_vis_dp, pos_tac_sp]):
    ax.bar(i, sum(~group)/1539, color = '#f03b20', width = .85)
    ax.bar(i, sum(group)/1539, bottom = sum(~group)/1539, color = '#2ca25f', width = .85)
    ax.bar(i, (1539-len(group))/1539, bottom = len(group)/1539, color = [0.5,0.5,0.5], width = .85)
    
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.xticks(rotation = 45, ha = 'right')
ax.set_xticks([0,1,2])
ax.set_xticklabels(['Neurons w/\ntactile DP', 'Neurons w/\nvisual DP', 'Neurons w/\ntactile SP'])
ax.text(.9, 0.62, 'No mod.', color = [0.5,0.5,0.5], transform = ax.transAxes)
ax.text(.9, 0.5, 'Excited', color ='#2ca25f', transform = ax.transAxes)
ax.text(.9, 0.38, 'Inhibited', color = '#f03b20', transform = ax.transAxes)
ax.set_ylabel('Fraction of neurons')
ax.set_xlim(-0.7,2.9)
fig.savefig('directional_mod_bar.pdf', format = 'pdf', dpi = 900)

In [ ]:
[sum(group) for group in [pos_tac_dp, pos_vis_dp, pos_tac_sp]]

In [ ]:
pd.concat([pos_tac_dp, pos_vis_dp, pos_tac_sp], axis = 1).any(axis = 1).sum()

In [ ]:
[sum(~group) for group in [pos_tac_dp, pos_vis_dp, pos_tac_sp]]

In [ ]:
pd.concat([~pos_tac_dp, ~pos_vis_dp, ~pos_tac_sp], axis = 1).any(axis = 1).sum()